## Colorful Image Colorization

Richard Zhang, Phillip Isola, Alexei A. Efros

year: 2016



### Importing libraries


In [1]:
import cv2
import numpy as np

### Load the model

In [3]:
prototxt_path = 'colorization_deploy_v2.prototxt' 
caffemodel_path = 'colorization_release_v2.caffemodel' # Change accordingly
npy_path = 'pts_in_hull.npy' 
model = cv2.dnn.readNetFromCaffe(prototxt_path, 
                                caffemodel_path)
c_points = np.load(npy_path)
c_points = c_points.T.reshape(2, 313, 1, 1).astype('float32')
model.getLayer(model.getLayerId('class8_ab')).blobs = [c_points]

model.getLayer(model.getLayerId('conv8_313_rh')).blobs = [np.full([1, 313], 2.606, dtype='float32')]

In [4]:
model.getLayerNames()

('conv1_1',
 'relu1_1',
 'conv1_2',
 'relu1_2',
 'conv1_2norm',
 'conv2_1',
 'relu2_1',
 'conv2_2',
 'relu2_2',
 'conv2_2norm',
 'conv3_1',
 'relu3_1',
 'conv3_2',
 'relu3_2',
 'conv3_3',
 'relu3_3',
 'conv3_3norm',
 'conv4_1',
 'relu4_1',
 'conv4_2',
 'relu4_2',
 'conv4_3',
 'relu4_3',
 'conv4_3norm',
 'conv5_1',
 'relu5_1',
 'conv5_2',
 'relu5_2',
 'conv5_3',
 'relu5_3',
 'conv5_3norm',
 'conv6_1',
 'relu6_1',
 'conv6_2',
 'relu6_2',
 'conv6_3',
 'relu6_3',
 'conv6_3norm',
 'conv7_1',
 'relu7_1',
 'conv7_2',
 'relu7_2',
 'conv7_3',
 'relu7_3',
 'conv7_3norm',
 'conv8_1',
 'relu8_1',
 'conv8_2',
 'relu8_2',
 'conv8_3',
 'relu8_3',
 'conv8_313',
 'conv8_313_rh',
 'class8_313_rh',
 'class8_ab',
 'Silence')

In [6]:
model.getLayersShapes((1, 1, 224, 224))
# layersIds, inLayersShapes, outLayersShapes

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56]),
 ((array([  1,   1, 224, 224]),),
  (array([  1,   1, 224, 224]),),
  (array([  1,  64, 224, 224]),),
  (array([  1,  64, 224, 224]),),
  (array([  1,  64, 112, 112]),),
  (array([  1,  64, 112, 112]),),
  (array([  1,  64, 112, 112]),),
  (array([  1, 128, 112, 112]),),
  (array([  1, 128, 112, 112]),),
  (array([  1, 128,  56,  56]),),
  (array([  1, 128,  56,  56]),),
  (array([  1, 128,  56,  56]),),
  (array([  1, 256,  56,  56]),),
  (array([  1, 256,  56,  56]),),
  (array([  1, 256,  56,  56]),),
  (array([  1, 256,  56,  56]),),
  (array([  1, 256,  28,  28]),),
  (array([  1, 256,  28,  28]),),
  (array([  1, 256,  28,  28]),),
  (array([  1, 512,  28,  28]),),
  (array([  1, 512,  28,  28]),),
  (array([  1, 512,  28,

### 1-Read an image, 2-Scale it, 3-Convert to Lab, 4-Resize
###  5- Pass image to the network, 6- Process the ab channel 7- Display the result

In [18]:
cv2.namedWindow('img', cv2.WINDOW_NORMAL)
for i in range(1, 7):
    img_path = 'a' + str(i) + '.jpg'
    img = cv2.imread(img_path)
    img_scale = img.astype('float32') / 255
    img_lab = cv2.cvtColor(img_scale, cv2.COLOR_BGR2LAB)
    L = img_lab[:, :, 0]
    model.setInput(cv2.dnn.blobFromImage(L, mean=50, size=(224, 224)))
    # (1, 1, 224, 224)
    ab = model.forward()
    # (1, 2, 56, 56)
    c = np.moveaxis(ab.squeeze(), [0], [2])
    # (56, 56, 2)
    ab_resized = cv2.resize(c, (img.shape[1], img.shape[0]))
    colored_lab = cv2.merge([L, ab_resized])
    colored_bgr = cv2.cvtColor(colored_lab, cv2.COLOR_LAB2BGR)
    colored_bgr = np.clip(colored_bgr, 0, 1)
    colored_bgr = (255 * colored_bgr).astype('uint8')
    stack = np.hstack((colored_bgr, img))
    cv2.imshow('img', stack)
    if cv2.waitKey(0) == ord('q'):
        break
cv2.destroyAllWindows()